In [1]:
import numpy as np
import pandas as pd

%load_ext rpy2.ipython
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import os

def load_smooth_dev():
    """Load the smooth package in development mode, avoiding package install cache"""
    smooth_path = "/home/filtheo/smooth/"
    
    ro.r(f'''
    if (!requireNamespace("devtools", quietly=TRUE)) {{
        install.packages("devtools", repos="https://cran.rstudio.com/", INSTALL_opts="--no-multiarch --no-test-load --no-save --no-restore --no-cache")
    }}
    devtools::load_all("{smooth_path}")
    ''')
    
    print("Smooth package loaded in development mode")

def install_and_load_smooth():
    """Install and load the smooth package from CRAN if not already installed"""
    ro.r('''
    if (!requireNamespace("smooth", quietly=TRUE)) {
        install.packages("smooth", repos="https://cran.rstudio.com/")
    }
    library(smooth)
    ''')
    print("Smooth package installed (if needed) and loaded from CRAN")

#load_smooth_dev()
install_and_load_smooth()

R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v2.0.6 loaded.


R[write to console]: This is package "smooth", v4.4.0.41005


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages



Smooth package installed (if needed) and loaded from CRAN


In [2]:
%%R

#library(smooth)
library(Mcomp)
m1_monthly <- subset(M1, "MONTHLY")



Loading required package: forecast
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 


In [3]:
import argparse
import time
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
from utils_benchmark import (
    compute_all_metrics,
    get_seasonality,
    list_available_datasets,
    load_dataset,
    print_available_datasets,
)

warnings.filterwarnings('ignore')
from smooth.adam_general.core.adam import ADAM

In [4]:
DEFAULT_DATA_DIR = "/home/filtheo/smooth/python/tests/speed_tests/benchmark_data"

In [5]:
def list_available_datasets(data_dir=None):
    """List all available datasets in the data directory."""
    data_dir = Path(data_dir) if data_dir else DEFAULT_DATA_DIR

    if not data_dir.exists():
        return []

    datasets = []
    for path in data_dir.iterdir():
        if path.is_dir() and (path / "metadata.csv").exists():
            datasets.append(path.name)

    return sorted(datasets)

list_available_datasets(DEFAULT_DATA_DIR)

['m1_monthly',
 'm1_quarterly',
 'm1_yearly',
 'm3_monthly',
 'm3_other',
 'm3_quarterly',
 'm3_yearly',
 'tourism_monthly',
 'tourism_quarterly',
 'tourism_yearly']

In [6]:
def load_and_datetime_index_series(dataset_path, 
                                   series, h, 
                                   train_start_date='2023-01-01'):
        """
        Loads train and test dataframes for a series, assigning a monthly datetime index starting at train_start_date.

        Args:
            dataset_path (str): Path to dataset directory.
            series (str): Series ID.
            h (int): Forecast horizon (length of test set).
            train_start_date (str): Date string for first train index (default '2023-01-01').

        Returns:
            tuple (train_df, test_df)
                train_df: Pandas DataFrame with datetime index.
                test_df: Pandas DataFrame with datetime index.
        """
        train_path = dataset_path + "/" + series + "_train.csv"
        test_path = dataset_path + "/" + series + "_test.csv"

        train_df = pd.read_csv(train_path)
        total_length = len(train_df)
        # As we have no real date info, just generate a fixed date range starting at Jan 2000
        dates = pd.date_range(start=train_start_date, periods=total_length, freq='M')
        train_df['t'] = dates
        end_date = train_df['t'].iloc[-1]
        train_df.set_index('t', inplace=True)
        test_df = pd.read_csv(test_path)
        # Use DateOffset for months to advance to first test period
        dates = pd.date_range(start=end_date + pd.DateOffset(months=1), periods=h, freq='M')
        test_df['t'] = dates
        test_df.set_index('t', inplace=True)
        return train_df, test_df

In [7]:
dataset = 'm1_monthly'
dataset_path = DEFAULT_DATA_DIR + "/" + dataset


metadata = pd.read_csv(dataset_path + "/metadata.csv")
series_ids = metadata['series_id'].tolist()
h = metadata['horizon'].unique()[0]
freq = metadata['frequency'].unique()[0]

In [8]:
# For every series
for series in series_ids:
    # Load and prepare series
    train_df, test_df = load_and_datetime_index_series(dataset_path, series, h)

    # define model 
    model_optimal = ADAM(model='ZZZ', lags=[freq], initial='optimal')
    model_optimal.fit(train_df)
    forecast_result = model_optimal.predict(h=h)
    forecast_result.index = test_df.index
    forecast_result['True'] = test_df['y']
    forecast_result['series_id'] = series
    print("Series: ", series)
    #print('--'*30)
    print(model_optimal)
    print('-'*100)
    break

Series:  series_0001
Time elapsed: 1.04 seconds
Model estimated using ADAM() function: ETS(MMN)
With optimal initialisation
Distribution assumed in the model: Gamma
Loss function type: likelihood; Loss function value: 595.9333
Persistence vector g:
 alpha   beta
0.0004 0.0002
Sample size: 42
Number of estimated parameters: 5
Number of degrees of freedom: 37
Information criteria:
      AIC      AICc       BIC      BICc
1201.8666 1203.5332 1210.5549 1213.6696
----------------------------------------------------------------------------------------------------


In [9]:
series_id = 6
series = series_ids[series_id]

train_df, test_df = load_and_datetime_index_series(dataset_path, series, h)
model_optimal = ADAM(model='ZZZ', lags=[freq], initial='optimal')
model_optimal.fit(train_df)
forecast_result = model_optimal.predict(h=h)
forecast_result.index = test_df.index
forecast_result['True'] = test_df['y']
forecast_result['series_id'] = series
print("Series: ", series)
#print('--'*30)
print(model_optimal)
print('-'*100)

Series:  series_0007
Time elapsed: 1.08 seconds
Model estimated using ADAM() function: ETS(MNM)
With optimal initialisation
Distribution assumed in the model: Gamma
Loss function type: likelihood; Loss function value: 457.8278
Persistence vector g:
 alpha  gamma
0.5099 0.0038
Damping parameter: 1.0000
Sample size: 42
Number of estimated parameters: 15
Number of degrees of freedom: 27
Information criteria:
      AIC      AICc       BIC      BICc
 945.6556  964.1171  971.7206 1006.2222
----------------------------------------------------------------------------------------------------


In [10]:
train_df

,y
t,
2023-01-31,117497
2023-02-28,153276
2023-03-31,241443
2023-04-30,218709
2023-05-31,202896
2023-06-30,200064
2023-07-31,192762
2023-08-31,109974
2023-09-30,154109


In [11]:
# Examine series_id from 0 to 9
print("="*80)
print("EXAMINING SERIES_ID FROM 0 TO 9")
print("="*80)

for series_id in range(10):  # 0 to 9
    print(f"\n{'='*80}")
    print(f"Series ID: {series_id}")
    print(f"{'='*80}")
    
    series = series_ids[series_id]
    train_df, test_df = load_and_datetime_index_series(dataset_path, series, h)
    
    model_optimal = ADAM(model='ZZZ', lags=[freq], initial='optimal')
    model_optimal.fit(train_df)
    forecast_result = model_optimal.predict(h=h)
    forecast_result.index = test_df.index
    forecast_result['True'] = test_df['y']
    forecast_result['series_id'] = series
    
    print(f"Series: {series}")
    print(model_optimal)
    print('-'*80)

EXAMINING SERIES_ID FROM 0 TO 9

Series ID: 0
Series: series_0001
Time elapsed: 0.85 seconds
Model estimated using ADAM() function: ETS(MMN)
With optimal initialisation
Distribution assumed in the model: Gamma
Loss function type: likelihood; Loss function value: 595.9333
Persistence vector g:
 alpha   beta
0.0004 0.0002
Sample size: 42
Number of estimated parameters: 5
Number of degrees of freedom: 37
Information criteria:
      AIC      AICc       BIC      BICc
1201.8666 1203.5332 1210.5549 1213.6696
--------------------------------------------------------------------------------

Series ID: 1
Series: series_0002
Time elapsed: 0.93 seconds
Model estimated using ADAM() function: ETS(MNN)
With optimal initialisation
Distribution assumed in the model: Gamma
Loss function type: likelihood; Loss function value: 422.5697
Persistence vector g:
 alpha
1.0000
Sample size: 109
Number of estimated parameters: 3
Number of degrees of freedom: 106
Information criteria:
      AIC      AICc       BIC

In [12]:
# Python: Loop through all series and save model type and loss to dataframe
results_python = []

print("Running Python models on all Mcomp series...")
print(f"Total series: {len(series_ids)}")
print("="*80)

total_series = 100
series_ids_temp = series_ids[:total_series]
for idx, series in enumerate(series_ids_temp):
    try:
        # Load and prepare series
        train_df, test_df = load_and_datetime_index_series(dataset_path, series, h)
        
        # Fit model
        model_optimal = ADAM(model='ZZZ', lags=[freq], initial='optimal')
        model_optimal.fit(train_df)
        
        # Extract model type and loss value
        model_type = model_optimal.model if hasattr(model_optimal, 'model') else 'Unknown'
        loss_value = model_optimal.adam_estimated.get('CF_value', None) if hasattr(model_optimal, 'adam_estimated') and model_optimal.adam_estimated else None
        
        # Save results (using idx+1 to match R's 1-based indexing)
        results_python.append({
            'index': idx + 1,  # 1-based index to match R
            'series_id': series,
            'model_type': model_type,
            'loss_value': loss_value
        })
        
        if (idx + 1) % 10 == 0:
            print(f"Processed {idx + 1}/{len(series_ids)} series...")
            
    except Exception as e:
        print(f"Error processing {series}: {e}")
        results_python.append({
            'index': idx + 1,
            'series_id': series,
            'model_type': 'Error',
            'loss_value': None
        })

# Create dataframe
df_python = pd.DataFrame(results_python)
print(f"\nCompleted! Processed {len(df_python)} series.")
print(f"\nPython Results Summary:")
print(df_python.head(10))
df_python

Running Python models on all Mcomp series...
Total series: 617


Processed 10/617 series...
Processed 20/617 series...
Processed 30/617 series...
Processed 40/617 series...
Processed 50/617 series...
Processed 60/617 series...
Processed 70/617 series...
Processed 80/617 series...
Processed 90/617 series...
Processed 100/617 series...

Completed! Processed 100 series.

Python Results Summary:
   index    series_id model_type   loss_value
0      1  series_0001   ETS(MMN)   595.933280
1      2  series_0002   ETS(MNN)   422.569677
2      3  series_0003   ETS(MAM)   718.085635
3      4  series_0004   ETS(ANA)   513.417968
4      5  series_0005   ETS(ANA)   333.333340
5      6  series_0006   ETS(ANN)   358.220959
6      7  series_0007   ETS(MNM)   457.827777
7      8  series_0008   ETS(ANN)   293.035531
8      9  series_0009   ETS(MNM)   332.679918
9     10  series_0010   ETS(MNM)  1105.388914


,index,series_id,model_type,loss_value
0,1,series_0001,ETS(MMN),595.933280
1,2,series_0002,ETS(MNN),422.569677
2,3,series_0003,ETS(MAM),718.085635
3,4,series_0004,ETS(ANA),513.417968
4,5,series_0005,ETS(ANA),333.333340
...,...,...,...,...
95,96,series_0096,ETS(MAA),378.933488
96,97,series_0097,ETS(MAN),265.380640
97,98,series_0098,ETS(MNN),488.908294
98,99,series_0099,ETS(MNN),532.663161


In [13]:
%%R
# R: Loop through all series and save model type and loss to dataframe
library(Mcomp)
m1_monthly <- subset(M1, "MONTHLY")

results_r <- data.frame(
  series_id = integer(),
  model_type = character(),
  loss_value = numeric(),
  stringsAsFactors = FALSE
)

cat("Running R models on all Mcomp series...\n")
cat(sprintf("Total series: %d\n", length(m1_monthly)))
cat("================================================================================\n")

total_series = 100
series_ids_temp = 1:total_series
for (series_id in series_ids_temp) {
  tryCatch({
    temp_series <- m1_monthly[[series_id]]
    model_opt <- adam(temp_series, model = "ZZZ", lags = c(12), initial = 'optimal')
    
    # Extract model type and loss value
    model_type <- model_opt$model
    loss_value <- model_opt$lossValue
    
    # Add to results
    results_r <- rbind(results_r, data.frame(
      series_id = series_id,
      model_type = model_type,
      loss_value = loss_value,
      stringsAsFactors = FALSE
    ))
    
    if (series_id %% 10 == 0) {
      cat(sprintf("Processed %d/%d series...\n", series_id, length(m1_monthly)))
    }
  }, error = function(e) {
    cat(sprintf("Error processing series %d: %s\n", series_id, conditionMessage(e)))
    results_r <<- rbind(results_r, data.frame(
      series_id = series_id,
      model_type = "Error",
      loss_value = NA,
      stringsAsFactors = FALSE
    ))
  })
}

cat(sprintf("\nCompleted! Processed %d series.\n", nrow(results_r)))
cat("\nR Results Summary:\n")
print(head(results_r, 10))
results_r

Running R models on all Mcomp series...
Total series: 617


Processed 10/617 series...
Processed 20/617 series...
Processed 30/617 series...
Processed 40/617 series...
Processed 50/617 series...
Processed 60/617 series...
Processed 70/617 series...
Processed 80/617 series...
Processed 90/617 series...
Processed 100/617 series...

Completed! Processed 100 series.

R Results Summary:
   series_id model_type loss_value
1          1   ETS(MMN)   595.9333
2          2   ETS(MNN)   422.5697
3          3   ETS(MAM)   718.0855
4          4   ETS(ANA)   513.4180
5          5   ETS(ANA)   333.5017
6          6   ETS(ANN)   358.2210
7          7   ETS(MNM)   457.8278
8          8   ETS(ANN)   293.0355
9          9   ETS(MNM)   332.6799
10        10   ETS(MNM)  1105.3890
    series_id model_type loss_value
1           1   ETS(MMN)   595.9333
2           2   ETS(MNN)   422.5697
3           3   ETS(MAM)   718.0855
4           4   ETS(ANA)   513.4180
5           5   ETS(ANA)   333.5017
6           6   ETS(ANN)   358.2210
7           7   ETS(MNM)   457.8278
8 

In [14]:
%%R -o results_r
results_r

    series_id model_type loss_value
1           1   ETS(MMN)   595.9333
2           2   ETS(MNN)   422.5697


3           3   ETS(MAM)   718.0855
4           4   ETS(ANA)   513.4180
5           5   ETS(ANA)   333.5017
6           6   ETS(ANN)   358.2210
7           7   ETS(MNM)   457.8278
8           8   ETS(ANN)   293.0355
9           9   ETS(MNM)   332.6799
10         10   ETS(MNM)  1105.3890
11         11   ETS(MNM)   911.2899
12         12   ETS(MNM)   280.2394
13         13   ETS(ANM)   253.4946
14         14   ETS(MAM)   475.5781
15         15   ETS(MNN)   909.7876
16         16   ETS(MNN)   335.0144
17         17   ETS(ANN)   252.2442
18         18   ETS(MNN)   839.1647
19         19   ETS(ANA)   613.8929
20         20   ETS(ANA)   573.2506
21         21   ETS(ANA)   563.4756
22         22   ETS(ANA)   457.7693
23         23   ETS(ANN)   145.3423
24         24   ETS(ANN)   328.2269
25         25   ETS(MNM)   522.9674
26         26   ETS(MAM)   496.4409
27         27   ETS(MAN)   479.0910
28         28   ETS(AAN)   475.2184
29         29   ETS(ANN)   343.4911
30         30   ETS(ANA)   

In [15]:
# Merge dataframes for comparison using index (1-based to match R)
results_r_py = results_r.copy()

# Merge on index (R's series_id is 1-based, matching our Python index)
df_comparison = df_python.merge(
    results_r_py,
    left_on='index',
    right_on='series_id',
    suffixes=('_python', '_r'),
    how='outer'
)

# Add comparison columns
df_comparison['model_type_match'] = df_comparison['model_type_python'] == df_comparison['model_type_r']
df_comparison['loss_diff'] = df_comparison['loss_value_python'] - df_comparison['loss_value_r']
df_comparison['loss_diff_abs'] = df_comparison['loss_diff'].abs()

print("="*80)
print("COMPARISON RESULTS")
print("="*80)
print(f"\nTotal series compared: {len(df_comparison)}")
print(f"Model types match: {df_comparison['model_type_match'].sum()}/{len(df_comparison)}")
if df_comparison['loss_diff_abs'].notna().any():
    print(f"Average absolute loss difference: {df_comparison['loss_diff_abs'].mean():.6f}")
    print(f"Max absolute loss difference: {df_comparison['loss_diff_abs'].max():.6f}")

print("\n" + "="*80)
print("First 20 comparison results:")
print("="*80)
display_cols = ['index', 'series_id_python', 'model_type_python', 'model_type_r', 'model_type_match', 
                'loss_value_python', 'loss_value_r', 'loss_diff_abs']
available_cols = [col for col in display_cols if col in df_comparison.columns]
print(df_comparison[available_cols].head(20))

print("\n" + "="*80)
print("Series with mismatched model types:")
print("="*80)
mismatched = df_comparison[~df_comparison['model_type_match']]
if len(mismatched) > 0:
    print(mismatched[['index', 'series_id_python', 'model_type_python', 'model_type_r']])
else:
    print("All model types match!")

df_comparison

COMPARISON RESULTS

Total series compared: 100
Model types match: 94/100
Average absolute loss difference: 0.356576
Max absolute loss difference: 12.464253

First 20 comparison results:
    index series_id_python model_type_python model_type_r  model_type_match  \
0       1      series_0001          ETS(MMN)     ETS(MMN)              True   
1       2      series_0002          ETS(MNN)     ETS(MNN)              True   
2       3      series_0003          ETS(MAM)     ETS(MAM)              True   
3       4      series_0004          ETS(ANA)     ETS(ANA)              True   
4       5      series_0005          ETS(ANA)     ETS(ANA)              True   
5       6      series_0006          ETS(ANN)     ETS(ANN)              True   
6       7      series_0007          ETS(MNM)     ETS(MNM)              True   
7       8      series_0008          ETS(ANN)     ETS(ANN)              True   
8       9      series_0009          ETS(MNM)     ETS(MNM)              True   
9      10      series_00

,index,series_id_python,model_type_python,loss_value_python,series_id_r,model_type_r,loss_value_r,model_type_match,loss_diff,loss_diff_abs
0,1,series_0001,ETS(MMN),595.933280,1,ETS(MMN),595.933280,True,-4.547474e-13,4.547474e-13
1,2,series_0002,ETS(MNN),422.569677,2,ETS(MNN),422.569677,True,-4.490630e-12,4.490630e-12
2,3,series_0003,ETS(MAM),718.085635,3,ETS(MAM),718.085548,True,8.710808e-05,8.710808e-05
3,4,series_0004,ETS(ANA),513.417968,4,ETS(ANA),513.417968,True,-1.136868e-13,1.136868e-13
4,5,series_0005,ETS(ANA),333.333340,5,ETS(ANA),333.501720,True,-1.683802e-01,1.683802e-01
...,...,...,...,...,...,...,...,...,...,...
95,96,series_0096,ETS(MAA),378.933488,96,ETS(MAA),378.933488,True,-1.193712e-12,1.193712e-12
96,97,series_0097,ETS(MAN),265.380640,97,ETS(MAN),265.380640,True,0.000000e+00,0.000000e+00
97,98,series_0098,ETS(MNN),488.908294,98,ETS(MNN),488.908294,True,1.193712e-11,1.193712e-11
98,99,series_0099,ETS(MNN),532.663161,99,ETS(MNN),532.663161,True,-2.614797e-12,2.614797e-12


In [19]:
df_comparison[df_comparison['model_type_match'] == False]

,index,series_id_python,model_type_python,loss_value_python,series_id_r,model_type_r,loss_value_r,model_type_match,loss_diff,loss_diff_abs
49,50,series_0050,ETS(MNM),1011.135750,50,ETS(MAM),1008.729678,False,2.406072,2.406072
51,52,series_0052,ETS(MAA),509.381366,52,ETS(ANA),521.845619,False,-12.464253,12.464253
62,63,series_0063,ETS(AAdN),492.682371,63,ETS(MMdN),490.574621,False,2.107750,2.107750
63,64,series_0064,ETS(MAdM),764.879758,64,ETS(MAM),765.738800,False,-0.859042,0.859042
64,65,series_0065,ETS(MNM),1016.422345,65,ETS(MAM),1015.152829,False,1.269516,1.269516
88,89,series_0089,ETS(MNM),988.509784,89,ETS(MAM),985.964382,False,2.545403,2.545403


In [20]:
indices = df_comparison[df_comparison['loss_diff'].abs() > 1]['index'].tolist()
df_comparison[df_comparison['loss_diff'].abs() > 1]

,index,series_id_python,model_type_python,loss_value_python,series_id_r,model_type_r,loss_value_r,model_type_match,loss_diff,loss_diff_abs
49,50,series_0050,ETS(MNM),1011.135750,50,ETS(MAM),1008.729678,False,2.406072,2.406072
51,52,series_0052,ETS(MAA),509.381366,52,ETS(ANA),521.845619,False,-12.464253,12.464253
62,63,series_0063,ETS(AAdN),492.682371,63,ETS(MMdN),490.574621,False,2.107750,2.107750
64,65,series_0065,ETS(MNM),1016.422345,65,ETS(MAM),1015.152829,False,1.269516,1.269516
65,66,series_0066,ETS(MAM),783.134521,66,ETS(MAM),778.513982,True,4.620539,4.620539
80,81,series_0081,ETS(MNM),961.804560,81,ETS(MNM),960.326524,True,1.478036,1.478036
83,84,series_0084,ETS(MAM),829.968647,84,ETS(MAM),826.444891,True,3.523756,3.523756
88,89,series_0089,ETS(MNM),988.509784,89,ETS(MAM),985.964382,False,2.545403,2.545403


In [21]:
indices

[50, 52, 63, 65, 66, 81, 84, 89]

In [22]:
indices = [50, 52, 63, 65, 66, 81, 84, 89]
for idx in indices:
    series = series_ids[idx-1]
    train_df, test_df = load_and_datetime_index_series(dataset_path, series, h)
    
    model_optimal = ADAM(model='ZZZ', lags=[freq], initial='optimal')
    model_optimal.fit(train_df)
    forecast_result = model_optimal.predict(h=h)
    forecast_result.index = test_df.index
    forecast_result['True'] = test_df['y']
    forecast_result['series_id'] = series
    
    print(f"Series: {series}")
    print(model_optimal)
    print('-'*80)

Series: series_0050
Time elapsed: 2.32 seconds
Model estimated using ADAM() function: ETS(MNM)
With optimal initialisation
Distribution assumed in the model: Gamma
Loss function type: likelihood; Loss function value: 1011.1358
Persistence vector g:
 alpha  gamma
0.4601 0.0063
Damping parameter: 1.0000
Sample size: 104
Number of estimated parameters: 15
Number of degrees of freedom: 89
Information criteria:
      AIC      AICc       BIC      BICc
2052.2715 2057.7260 2091.9374 2104.6039
--------------------------------------------------------------------------------
Series: series_0052
Time elapsed: 3.72 seconds
Model estimated using ADAM() function: ETS(MAA)
With optimal initialisation
Distribution assumed in the model: Gamma
Loss function type: likelihood; Loss function value: 509.3814
Persistence vector g:
 alpha   beta  gamma
0.9837 0.0392 0.0091
Sample size: 101
Number of estimated parameters: 17
Number of degrees of freedom: 84
Information criteria:
      AIC      AICc       BIC   

In [23]:
%%R -i indices
for (series_id in indices) {
  cat("\n")
  cat("================================================================================\n")
  cat(sprintf("Series ID: %d\n", series_id))
  cat("================================================================================\n")
  
  temp_series <- m1_monthly[[series_id]]
  model_opt <- adam(temp_series, model = "ZZZ", lags = c(12), initial = 'optimal')
  print(model_opt)
  cat("--------------------------------------------------------------------------------\n")
}


Series ID: 50


Time elapsed: 4 seconds
Model estimated using adam() function: ETS(MAM)
With optimal initialisation
Distribution assumed in the model: Gamma
Loss function type: likelihood; Loss function value: 1008.73
Persistence vector g:
 alpha   beta  gamma 
0.5477 0.0101 0.0012 

Sample size: 104
Number of estimated parameters: 17
Number of degrees of freedom: 87
Information criteria:
     AIC     AICc      BIC     BICc 
2051.459 2058.576 2096.414 2112.939 

Forecast errors:
ME: -2386.61; MAE: 2694.911; RMSE: 4113.797
sCE: -126.034%; Asymmetry: -85.3%; sMAE: 7.906%; sMSE: 1.457%
MASE: 0.373; RMSSE: 0.445; rMAE: 0.144; rRMSE: 0.212
--------------------------------------------------------------------------------

Series ID: 52
Time elapsed: 1.05 seconds
Model estimated using adam() function: ETS(ANA)
With optimal initialisation
Distribution assumed in the model: Normal
Loss function type: likelihood; Loss function value: 521.8456
Persistence vector g:
 alpha  gamma 
0.8996 0.0996 

Sample size: 101
